In [1]:
%load_ext autoreload
%autoreload 2

import IPython
from pathlib import Path
import os
locals = IPython.extract_module_locals() # type: ignore
notebook_name = "/".join(locals[1]["__vsc_ipynb_file__"].split("/"))
os.chdir(Path(notebook_name).parent.parent)

In [29]:
from retail_recommender_system.utils import batch_dict_to_device, split_by_time, filter_set
import polars as pl
from torch.utils.data import DataLoader
from datetime import datetime
from retail_recommender_system.models.deepfm import *
from retail_recommender_system.models.mf import *
from retail_recommender_system.models.mfconv import *
from retail_recommender_system.data.loader import *
from retail_recommender_system.evaluation.metrics import *

from retail_recommender_system.models.mfconv import collate_fn

In [19]:
dataset_full = load_dataset(DataConfig(dataset="hm", prefix="full"))
dataset_full.load()

In [5]:
dataset = load_dataset(DataConfig(dataset="hm", prefix="frac_0_001"))
dataset.load()

In [20]:
train_relations, valid_relations = dataset_full.data["relations"]

In [28]:
train_relations

t_dat,article_id,price,sales_channel_id,customer_id,customer_id_map,article_id_map
date,i64,f64,i64,str,u32,u32
2018-09-20,671505002,0.033881,1,"""aa21cef1a12574e4ecec4dacd2bf6e…",557823,40494
2018-09-20,659459003,0.084729,2,"""e788c7ef1a2258b63380221c9ed045…",758402,37403
2018-09-20,666448004,0.025407,1,"""b13bbd34befe78cf06091823b73a6b…",581059,39456
2018-09-20,655567001,0.059305,1,"""b13bbd34befe78cf06091823b73a6b…",581059,36406
2018-09-20,568601006,0.050831,1,"""10112edae095691f69a9d7d8c6eea7…",52511,15241
…,…,…,…,…,…,…
2020-02-10,708021001,0.016695,2,"""4c32261f31191c29eb66ad00807841…",249415,52203
2020-02-10,797078010,0.03339,2,"""4c32261f31191c29eb66ad00807841…",249415,74620
2020-02-10,849257001,0.041746,2,"""4c32261f31191c29eb66ad00807841…",249415,81809


In [33]:
fltr = pl.concat([train_relations, valid_relations]).filter(pl.col("t_dat") > pl.lit(datetime(2020, 9, 1)))

train_cst = fltr["customer_id_map"].unique().to_numpy().ravel()
train_art = fltr["article_id_map"].unique().to_numpy().ravel()
print(fltr.shape, train_cst.shape, train_art.shape)

(190480, 7) (86074,) (10576,)


In [17]:
filtered_users = train_relations.group_by("customer_id").agg(pl.len()).filter(pl.col("len") >= 5).select("customer_id")
relations = train_relations.data["relations"].join(filtered_users, on="customer_id", how="right")

train_relations

AttributeError: 'DataFrame' object has no attribute 'data'

In [6]:
train_relations, valid_relations = dataset.data["relations"]

In [10]:
train_relations['article_id_map'].unique()

article_id_map
i64
0
1
2
3
4
…
12848
12849
12850


In [7]:
dataset.data["users"]

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,customer_id_map
str,f64,f64,str,str,i64,str,i64
"""00484a38eb52c45e130a5042121b66…",1.0,1.0,"""ACTIVE""","""Regularly""",47,"""bc8a8baa2a64604800736e62f27f6b…",0
"""006214f74ffd95b48aadf987598c67…",1.0,1.0,"""ACTIVE""","""Regularly""",23,"""c03914f412bcf4bf499b19ade9ba9f…",1
"""00641dddccb40cfd2254dae258361d…",null,null,"""ACTIVE""","""NONE""",19,"""2c29ae653a9282cce4151bd87643c9…",2
"""008e907334dd0209017ffdf316bcfe…",1.0,1.0,"""ACTIVE""","""Regularly""",39,"""fd9a937d4c024f1e0ba721d5563009…",3
"""00ad45cc77f793647d5531f3d6cd25…",null,null,"""ACTIVE""","""NONE""",21,"""21004dc15a55e42444b8b334d100e0…",4
…,…,…,…,…,…,…,…
"""fe800124498091996dfa9e9e6c28c1…",null,null,"""ACTIVE""","""NONE""",39,"""902cd966379071db2b727625adbf0f…",1145
"""fed63af5117f9fcbd7f437b917f862…",null,null,"""ACTIVE""","""NONE""",22,"""63c79a463a5e44bf5461283dfabdec…",1146
"""ff381ddc2199f27f79ebc9cc28bcdf…",null,null,"""ACTIVE""","""NONE""",null,"""06179c59b53b23a81838c8e9026a67…",1147


In [8]:
dataset.data["items"]

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,article_id_map,path
i64,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,str,i64,str
108775015,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…",0,""".data/hm/base/images/010/01087…"
108775044,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…",1,""".data/hm/base/images/010/01087…"
108775051,108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…",2,""".data/hm/base/images/010/01087…"
110065001,110065,"""OP T-shirt (Idro)""",306,"""Bra""","""Underwear""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1339,"""Clean Lingerie""","""B""","""Lingeries/Tights""",1,"""Ladieswear""",61,"""Womens Lingerie""",1017,"""Under-, Nightwear""","""Microfibre T-shirt bra with un…",3,""".data/hm/base/images/011/01100…"
111565001,111565,"""20 den 1p Stockings""",304,"""Underwear Tights""","""Socks & Tights""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",3608,"""Tights basic""","""B""","""Lingeries/Tights""",1,"""Ladieswear""",62,"""Womens Nightwear, Socks & Tigh""",1021,"""Socks and Tights""","""Semi shiny nylon stockings wit…",4,""".data/hm/base/images/011/01115…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
870282001,870282,"""Junia section speed""",259,"""Shirt""","""Garment Upper body""",1010016,"""Solid""",52,"""Pink""",7,"""Medium""",4,"""Pink""",1522,"""Blouse""","""A""","""Ladieswear""",1,"""Ladieswear""",15,"""Womens Everyday Collection""",1010,"""Blouses""","""Straight-cut blouse in a crisp…",12848,""".data/hm/base/images/087/08702…"
870804001,870804,"""Shadow shacket""",262,"""Jacket""","""Garment Upper body""",1010004,"""Check""",14,"""Dark Beige""",4,"""Dark""",11,"""Beige""",1244,"""Outdoor/Blazers""","""D""","""Divided""",2,"""Divided""",53,"""Divided Collection""",1007,"""Outdoor""","""Shirt jacket in woven fabric w…",12849,""".data/hm/base/images/087/08708…"
873915001,873915,"""New Pluto Dress""",265,"""Dress""","""Garment Full body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1343,"""Projects Dresses""","""D""","""Divided""",2,"""Divided""",50,"""Divided Projects""",1013,"""Dresses Ladies""","""Calf-length dress in crinkled …",12850,""".data/hm/base/images/087/08739…"


In [ ]:
train_relations["customer_i"]

In [ ]:
valid_relations

In [ ]:
train_cst = train_relations.select("customer_id_map").unique().to_numpy().ravel()
train_art = train_relations.select("article_id_map").unique().to_numpy().ravel()
print(train_cst.shape, train_art.shape)

valid_cst = valid_relations.select("customer_id_map").unique().to_numpy().ravel()
valid_art = valid_relations.select("article_id_map").unique().to_numpy().ravel()
print(valid_cst.shape, valid_art.shape)

import numpy as np
print(np.setdiff1d(valid_cst, train_cst).shape)
print(np.setdiff1d(valid_art, train_art).shape)

In [ ]:
mfconv_dataset = MFConvDataset(
    relations=dataset.data["relations"],
    users=dataset.data["users"],
    items=dataset.data["items"],
    namings=dataset.namings,
    neg_sampl=0,
    images=Path(".data/hm/intermediate/frac_0_01/images.pt")
)
mfconv_loader = DataLoader(mfconv_dataset, batch_size=256, shuffle=False, collate_fn=collate_fn)

model = MFConv(
    MFConvModelConfig(n_users=dataset.n_users, n_items=dataset.n_items, emb_size=16, )
)
#eval_dataset = MFConvEvalDataset(base_dataset=val_dataset, user_batch_size=5)

In [3]:
from torch.utils.data import DataLoader, TensorDataset
import torch

In [ ]:
def cfn(batch):
    return torch.tensor([b[0] for b in batch])

dl = DataLoader(TensorDataset(torch.arange(10)), batch_size=3, shuffle=False, drop_last=False, collate_fn=cfn)
iter_ = iter(dl)
print(next(iter_))

In [ ]:
next(iter_)[0]

In [ ]:
next(iter_)

In [42]:
model.precompute_img_embeddings(mfconv_dataset._images)

In [ ]:
model.recommend(torch.tensor([0, 1, 2, 3, 4]))

In [ ]:
#tensor_imgs = mfconv_dataset.read_imgs(mfconv_dataset.items_set(), tqdm_=True)

In [17]:
#torch.save(tensor_imgs, ".data/hm/intermediate/frac_0_01/images.pt")

In [ ]:
from tqdm import tqdm

for i in tqdm(mfconv_loader):
    print(i["i_img"].shape)

In [60]:
b = next(iter(val_loader))

In [ ]:
model.conv(b["i_img"])

In [ ]:
model(b)

In [ ]:
val_dataset[0]["i_img"][0].shape

In [101]:
from PIL import Image

# Replace 'path_to_image' with the actual path to your image file
image_path = '.data/hm/base/images/061/0616100001.jpg'
image = Image.open(image_path)
image.show()

In [ ]:
import numpy as np
np_image = np.array(image)
np_stack_image = np.stack((np_image,)*3, axis=-1)
plt.imshow(np.array(np_stack_image))

In [ ]:
from PIL import Image
import torch
import numpy as np

import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 10, figsize=(20, 5))
for i in range(10):
    ax[i].imshow(val_dataset[i]["i_img"][0].permute(1, 2, 0))
    ax[i].axis('off')
plt.show()

In [97]:
train_dataset = MFConvDataset(
    relations=dataset.data["relations"],
    users=dataset.data["users"],
    items=dataset.data["items"],
    namings=dataset.namings,
    neg_sampl=1,
)

In [98]:
loader = DataLoader(train_dataset, batch_size=3, shuffle=True, collate_fn=collate_fn)

In [99]:
iter_loader = iter(loader)

In [ ]:
model = MFConv(MFConvModelConfig(
    n_users=dataset.n_users, 
    n_items=dataset.n_items,
    emb_size=8
))
model(next(iter_loader))

In [54]:
df  =pl.DataFrame({"value": range(1, 101)})

In [ ]:
df[3:]